<a href="https://colab.research.google.com/github/navneet-g/sde-skills-genai-workshop/blob/main/Workshop_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-reqs

1. Bring a laptop or pair with someone to participate in hands-on lab
2. Signup for free google Colab at https://colab.research.google.com/
3. Get an api key at https://platform.openai.com/api-keys
4. We will follow the Notebook at https://github.com/navneet-g/sde-skills-genai-workshop/blob/main/Workshop_1.ipynb




# Guess the word


### Example 1
* He loved to study and when he grew up he became a <?>

### Example 2
* He wore a white coat and stethoscope to work everyday. He loved to study and when he grew up he became a <?>.




That is how transformers work, they guess the word based on the context.

* Large langugage models are trained on lots of data and can help and

In [1]:
!pip install --q -U langchain langchain_community \
openai langchain-openai selenium unstructured \
langchain-text-splitters unstructured faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain_community.document_loaders import SeleniumURLLoader  # loading documents
from langchain.text_splitter import CharacterTextSplitter  # splitting text
from langchain_community.vectorstores import (
    FAISS,
)  # creating vector store from embeddings; can use chromadb instead as well
from langchain.chains import RetrievalQA  # creating qa system
from google.colab import userdata

from langchain_openai import ChatOpenAI  # using llm for qa system
from langchain_openai import OpenAIEmbeddings  # embedding text with openai
import openai

In [3]:
openai_api_key=userdata.get('OPENAI_API_KEY')

# docs https://python.langchain.com/docs/integrations/llms/google_ai/
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", openai_api_key=openai_api_key)

In [ ]:
llm.invoke("When were the last academy awards").content

'The last Academy Awards, also known as the Oscars, were held on April 25, 2021.'

# **In context learning**

In [7]:
context = "Context: \n\
The 96th Academy Awards ceremony, which was presented by the Academy of Motion Picture Arts and Sciences (AMPAS), took place on March 10, 2024, at the Dolby Theatre in Hollywood, Los Angeles.[6] During the gala, the AMPAS presented Academy Awards (commonly referred to as Oscars) in 23 categories honoring films released in 2023. Comedian Jimmy Kimmel hosted the show for the fourth time.[a] \n"

question = "When were the 96th academy awards?"

llm.invoke(context + question)


AIMessage(content='The 96th Academy Awards took place on March 10, 2024.', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 113, 'total_tokens': 130}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-6b1f3544-4bfc-4e54-be36-7b1efd0cdb34-0')

In [5]:
question ="Who were the academy awards nominees"
llm.invoke(context + question)

AIMessage(content='The Academy Awards nominees for the 96th ceremony in 2024 would have included various actors, directors, writers, and other individuals involved in the films released in 2023. Some of the nominees could have included actors like Leonardo DiCaprio, Jennifer Lawrence, and Denzel Washington, directors like Steven Spielberg, Christopher Nolan, and Greta Gerwig, and writers like Aaron Sorkin, Quentin Tarantino, and Phoebe Waller-Bridge. The specific nominees in each category would have been announced closer to the date of the ceremony.', response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 110, 'total_tokens': 222}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-3f333e96-78cf-43fe-bf29-affffb247e80-0')

# **RAG**

In [ ]:
# load url
urls = [
    "https://en.wikipedia.org/wiki/96th_Academy_Awards",
]

loader = SeleniumURLLoader(urls=urls)
data = loader.load()


# split document by character
print("Splitting document by character...")
text_splitter = CharacterTextSplitter(
    separator="\n", chunk_size=1000, chunk_overlap=200
)

# split into multiple documents
print("Splitting into multiple documents...")
docs = text_splitter.split_documents(data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Splitting document by character...
Splitting into multiple documents...


In [ ]:
print("Creating vector store...")
# create vector store
db = FAISS.from_documents(docs, OpenAIEmbeddings( openai_api_key=openai_api_key))

Creating vector store...


In [ ]:

# create retriever to ask questions using openai and vector store
print("Creating retriever...")
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 10}),
)

Creating retriever...


In [ ]:
def ask_question(question):
    print("Query: " + question)
    print("Response: " + qa.invoke(question)['result'])
    print("===============")

In [ ]:
ask_question("Who were the academy awards winners?")
ask_question("What date did the academy awards happen?")
ask_question("What date did the 96th academy awards happen?")

Query: Who were the academy awards winners?
Response: The winners of the 96th Academy Awards included Oppenheimer, which won 7 awards, Poor Things, which won 4 awards, and The Zone of Interest, which won 2 awards. Specific categories and winners can be found in the full list of winners released by the Academy of Motion Picture Arts and Sciences.
Query: What date did the academy awards happen?
Response: The 96th Academy Awards ceremony took place on March 10, 2024, at the Dolby Theatre in Hollywood, Los Angeles.
Query: What date did the 96th academy awards happen?
Response: The 96th Academy Awards ceremony took place on March 10, 2024.
